In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

In [11]:
from modules.data.conll2003.prc import conll2003_preprocess

In [12]:
data_dir = "/home/eartemov/ae/work/conll2003/"

In [13]:
conll2003_preprocess(data_dir)

## IO markup

### Train

In [15]:
from modules.data import bert_data

In [25]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels.txt",
    clear_cache=True,
    model_name="bert-base-cased"
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [18]:
from modules.models.bert_models import BERTBiLSTMCRF

In [ ]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3)

In [27]:
from modules.train.train import NerLearner

In [28]:
num_epochs = 100

In [33]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-base-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [34]:
model.get_n_trainable_params()

2235023

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [2]:
from modules.data import bert_data

In [3]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels.txt",
    clear_cache=True,
    model_name="bert-base-cased"
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [4]:
from modules.models.bert_models import BERTBiLSTMCRF

In [5]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3)

In [6]:
from modules.train.train import NerLearner

In [8]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-base-IO.cpt")

In [9]:
model.get_n_trainable_params()

2235023

In [10]:
learner.load_model()

### Predict

In [11]:
from modules.data.bert_data import get_data_loader_for_predict

In [12]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [13]:
preds = learner.predict(dl)

In [14]:
from sklearn_crfsuite.metrics import flat_classification_report

In [15]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [16]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [17]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [18]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9504    0.9451    0.9477      2005
         I_O     0.9970    0.9969    0.9970     41651
      I_MISC     0.9372    0.9036    0.9201      1255
       I_PER     0.9852    0.9849    0.9851      2848
       I_LOC     0.9668    0.9688    0.9678      1922

   micro avg     0.9918    0.9907    0.9912     49681
   macro avg     0.9673    0.9599    0.9635     49681
weighted avg     0.9917    0.9907    0.9912     49681



In [19]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [ ]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

## BIO markup

### Train

In [3]:
from modules.data import bert_data

In [4]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels.txt",
    clear_cache=True,
    model_name="bert-base-cased", markup="BIO"
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMCRF

In [6]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3)

In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-base-BIO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

2236155

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [79]:
from modules.data import bert_data

In [12]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels.txt",
    clear_cache=False,
    model_name="bert-base-cased", markup="BIO"
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [13]:
from modules.models.bert_models import BERTBiLSTMCRF

In [14]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir /tmp/tmpb9shk2lo
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



In [15]:
from modules.train.train import NerLearner

In [16]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-base-BIO.cpt", t_total=num_epochs * len(data.train_dl))

In [17]:
model.get_n_trainable_params()

2236155

In [18]:
learner.load_model()

### Predict

In [19]:
from modules.data.bert_data import get_data_loader_for_predict

In [20]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [21]:
preds = learner.predict(dl)

In [22]:
from sklearn_crfsuite.metrics import flat_classification_report

In [23]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [24]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [25]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels,
                                           labels=["I_ORG", "I_PER", "I_LOC", "B_ORG", "B_PER", "B_LOC"], digits=4)

In [26]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9505    0.9243    0.9372       727
       I_PER     0.9914    0.9854    0.9884      1166
       I_LOC     0.9444    0.9261    0.9352       257
       B_ORG     0.9273    0.9280    0.9276      1278
       B_PER     0.9687    0.9744    0.9715      1682
       B_LOC     0.9621    0.9616    0.9619      1665

   micro avg     0.9603    0.9572    0.9588      6775
   macro avg     0.9574    0.9500    0.9536      6775
weighted avg     0.9603    0.9572    0.9587      6775



In [27]:
from modules.analyze_utils.plot_metrics import get_f1_score

In [28]:
get_f1_score(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC", "B_ORG", "B_PER", "B_LOC"])

0.9536399507613248

In [29]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [30]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.9135    0.9075    0.9105      1664
        MISC     0.8922    0.8695    0.8807       904
           O     0.9907    0.9917    0.9912     41632
         ORG     0.8498    0.8412    0.8455      1278
         PER     0.9495    0.9507    0.9501      1682

    accuracy                         0.9809     47160
   macro avg     0.9191    0.9121    0.9156     47160
weighted avg     0.9808    0.9809    0.9808     47160

